<a href="https://colab.research.google.com/github/Varsha-Sri-Chukka/23CSBTB50/blob/main/AIML_A3_2494.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1 Implement the AI Game Strategy
##Part 1 –(a). Install the Python Libraries required for Game Strategy

In [2]:
#1. Implement the MiniMax Search Algorithm
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools
cache=functools.lru_cache(10**6)

In [3]:
from functools import lru_cache

class Game:

    def actions(self, state):
        """Returns a list of actions available from the given state."""
        raise NotImplementedError

    def result(self, state, action):
        """Returns the result of applying an action to the given state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Returns True if the state is a terminal state, False otherwise."""
        raise NotImplementedError

    def utility(self, state):
        """Returns the utility of the terminal state."""
        raise NotImplementedError

In [4]:
def play_game(game, strategies: dict, verbose=False):
    """
    Play a turn-taking game.
    strategies is a {player name: function} dict, where function(state, game)
    is used to get the player's move.
    """
    state = game.initial_state()
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](state, game)
        state = game.result(state, move)
        if verbose:
            print('Player', player,  'move:' ,move)
            print(state)
    return state

##Part 2 – Implement the Game Strategy Algorithms.
##Implement the MiniMax Search Algorithm


In [5]:
import math

def minimax_search(game, state):
    """
    Search game tree to determine best move; return (value, move) pair.
    """
    player = state.to_move


    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None

        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state), None
        v , move = +infinity, None

        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)
    infinity = math.inf

##2. Implement the Alpha-Beta Search Algorithm

In [6]:
import math

def alphabeta_search(game, state):
    """
    Search game to determine best action; use alpha-beta pruning.
    ""Search all the way to leaves"""
    player = state.to_move
    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity,None

        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v , move = +infinity, None

        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move



    return max_value(state, -infinity, +infinity)

##Part 3 – Implement the Game Strategy using TicTocToe. [CO4]
##Implement TicToCToe game using init , actions, result, is terminal, utility, display constructors

In [7]:
class Game:
    """A base class for game-related methods and properties."""
    def actions(self, board):
        """Returns a set of legal actions for the given board."""
        raise NotImplementedError

    def result(self, board, action):
        """Returns the resulting board after applying the action."""
        raise NotImplementedError

    def utility(self, board, player):
        """Returns the utility value of the board for the given player."""
        raise NotImplementedError

    def is_terminal(self, board):
        """Returns True if the board is in a terminal state, False otherwise."""
        raise NotImplementedError

    def display(self, board):
        """Displays the board."""
        raise NotImplementedError

class TicTacToe(Game):
    """Play TicTacToe on a 'height' by 'width' board, needing 'k' in a row to win.
    'X' plays first against 'O'.
    """
    def _init_(self, height=3, width=3, k=3):
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        new_board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = self.k_in_row(new_board, player, square)
        new_board.utility = (0 if not win else +1 if player == 'X' else -1)
        return new_board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares - set(board)) == 0

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square):
        """True if player has k pieces in a line through square."""
        def in_row(x, y, dx, dy):
            count = 0
            while 0 <= x < board.width and 0 <= y < board.height and board.board.get((x, y)) == player:
                count += 1
                x += dx
                y += dy
            return count

        x, y = square
        return any(
            in_row(x, y, dx, dy) + in_row(x - dx, y - dy, -dx, -dy) - 1 >= self.k
            for dx, dy in [(0, 1), (1, 0), (1, 1), (1, -1)]
        )

In [8]:
from collections import defaultdict

class Board(defaultdict):
    """A board has the player to move, a cached utility value,
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'.
    """
    empty = '.'
    off = '#'

    def _init_(self, width=8, height=8, to_move=None, **kwds):
        super()._init_(self.empty)
        self.width = width
        self.height = height
        self.to_move = to_move
        self.utility = 0  # Initialize utility value
        self.update(kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        """Return the content at location or the off marker if out of bounds."""
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self[x, y]
        else:
            return self.off

    def _hash_(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def _repr_(self):
       def row(y):
            return ' '.join(self[x, y] for x in range(self.width))
       return '\n'.join(map(row, range(self.height))) + '\n'